<h2><font size = 5>Peer-graded Assignment:
Segmenting and Clustering Neighborhoods in Toronto</font></h2>

##### Import necessary Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import html5lib
from bs4 import BeautifulSoup
import folium
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import os
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


#### 2. Scrape the provided wiki page to obtain the list of postal codes
> `https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M`

#### Ensure ability to establish connection to the URL and receive a response

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postal_url = requests.get(url)
postal_url

<Response [200]>

#### Get relevant part of datafile and transform it into a *pandas* dataframe

In [4]:
postal_data = pd.read_html(postal_url.text)
len(postal_data), type(postal_data)

(3, list)

From above printout of length and type, there are 3 tables with a type list

In [5]:
# Pull the relevant datatable
postal_data = postal_data[0]
postal_data.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


<h4>Define information of interest and filter dataframe,
removing the Borough's that read <em>'Not assigned'</em></h3>

In [6]:
# keep only columns that include an assigned borough, and ignore if unassigned
filtered_columns = postal_data[~postal_data.Borough.str.contains('Not assigned')]
filtered_columns.head(12)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Reading the <em>Shape</em> of the dataframe to print the number of rows

In [7]:
filtered_columns.shape


(103, 3)

#### Create a geocoder function to loop through the postal codes and assign the correct longitude and latitude

In [8]:
import geocoder

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while lat_lng_coords is None :
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    return latitude, longitude

#### Read the cvs file data into a dataframe

In [9]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the latitude and longitude coordinates with the corresponding Postal Code in the initial dataset

In [10]:
merge_geo_data = pd.merge(geo_df, filtered_columns, on='Postal Code')
geo_data = merge_geo_data[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
geo_data.sort_index(kind='quicksort')
geo_data.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Obtain all the borough's in Toronto

In [11]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Define Foursquare Credentials and Version


In [12]:
CLIENT_ID = '23XKBQIHR43GO4QC5TUCSO2QLXJCLTVZDKRH5M4GIQXAS0HU' # your Foursquare ID
CLIENT_SECRET = 'Z2RELZ12VGKJFGAO0E2P1NDLNA2ZZSJIC4JRQNYE0R5PXHAU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentials:
CLIENT_ID: 23XKBQIHR43GO4QC5TUCSO2QLXJCLTVZDKRH5M4GIQXAS0HU
CLIENT_SECRET:Z2RELZ12VGKJFGAO0E2P1NDLNA2ZZSJIC4JRQNYE0R5PXHAU
